In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 명세서

### 학습 데이터 (user_id : 10000 ~ 24999, 15000명)


- train_err_data.csv : 시스템에 발생한 에러 로그

- train_quality_data.csv : 시스템 퀄리티 로그

- train_problem_data.csv : 사용자 불만 및 불만이 접수된 시간



### 테스트 데이터(user_id : 30000 ~ 44998, 14999명)


- test_err_data.csv : 시스템에 발생한 에러 로그

- test_quality_data.csv : 시스템 퀄리티 로그

- sample_submission.csv : 사용자 불만 확률(0~1) (제출용)

### 추가 데이터 설명

- err : 사용자가 시스템 작동시 시스템 로그 중에서 상태와 관련 있는 로그만을 정제하여 수집(예시, 시스템 연결 상태 및 시스템 강제 리붓 등)

- quality : 사용자의 시스템 작동 중 문제가 발생하면 측정 가능한 지표들로 해당 시점으로부터 2시간 단위 수집
<img align="left" src = "erd.png" width = 800>

# Data Loading

In [2]:
train_err = pd.read_csv('train_err_data.csv')
train_quality = pd.read_csv('train_quality_data.csv')
train_problem = pd.read_csv('train_problem_data.csv')
test_err = pd.read_csv('test_err_data.csv')
test_quality = pd.read_csv('test_quality_data.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
error = pd.concat([train_err, test_err])
quality = pd.concat([train_quality, test_quality])

In [4]:
# 데이터 세부사항
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

test_user_id_max = 44998
test_uset_id_min = 30000
test_user_number = 14999

# Feature Making

In [5]:
f_list = []
s_list = []

# Error Feature

### from model_nm

**[사용한 모델의 수]**

In [6]:
f = error.groupby('user_id')['model_nm'].agg([('model_nm_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,user_id,model_nm_cnt
0,10000,1
1,10001,1
2,10002,1
3,10003,1
4,10004,1
...,...,...
29993,44994,1
29994,44995,1
29995,44996,1
29996,44997,1


**[에러발생이 가장 많은 모델]**

In [7]:
s = error.groupby('user_id')['model_nm'].agg([('most_model', lambda x : x.value_counts().idxmax())]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

,user_id,most_model_model_0,most_model_model_1,most_model_model_2,most_model_model_3,most_model_model_4,most_model_model_5,most_model_model_6,most_model_model_7,most_model_model_8
0,10000,0,0,0,1,0,0,0,0,0
1,10001,0,0,1,0,0,0,0,0,0
2,10002,0,0,0,1,0,0,0,0,0
3,10003,0,0,1,0,0,0,0,0,0
4,10004,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,1,0,0,0,0,0,0,0
29994,44995,1,0,0,0,0,0,0,0,0
29995,44996,1,0,0,0,0,0,0,0,0
29996,44997,1,0,0,0,0,0,0,0,0


**[모델별 에러발생 횟수]**

In [8]:
s = pd.pivot_table(error, index = 'user_id', columns = 'model_nm', values = 'fwver', aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

model_nm,user_id,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8
0,10000,0,0,0,317,0,0,0,0,0
1,10001,0,0,2365,0,0,0,0,0,0
2,10002,0,0,0,306,0,0,0,0,0
3,10003,0,0,306,0,0,0,0,0,0
4,10004,777,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,1115,0,0,0,0,0,0,0
29994,44995,515,0,0,0,0,0,0,0,0
29995,44996,2233,0,0,0,0,0,0,0,0
29996,44997,24671,0,0,0,0,0,0,0,0


**[모델별 에러발생 비율]**

In [9]:
s = pd.pivot_table(error, index = 'user_id', columns = 'model_nm', values = 'fwver', aggfunc = 'count', fill_value = 0).reset_index()
s1 = s.iloc[:,1:].apply(lambda x : x/sum(x), axis = 1)
s2 = pd.concat([s.iloc[:,0], s1], axis = 1)
s_list.append(s2); display(s2)

,user_id,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8
0,10000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
29993,44994,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29994,44995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29995,44996,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,44997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### from fwver

**[사용한 펌웨어 버전 수]**

In [10]:
f = error.groupby('user_id')['fwver'].agg([('fwver_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,user_id,fwver_cnt
0,10000,1
1,10001,2
2,10002,1
3,10003,2
4,10004,2
...,...,...
29993,44994,1
29994,44995,2
29995,44996,2
29996,44997,2


**[에러발생이 가장 많은 펌웨어 버전]**

In [11]:
s = error.groupby('user_id')['fwver'].agg([('most_fwver', lambda x : x.value_counts().idxmax())]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

,user_id,most_fwver_03.11.1141,most_fwver_03.11.1149,most_fwver_03.11.1167,most_fwver_04.16.2641,most_fwver_04.16.3345,most_fwver_04.16.3439,most_fwver_04.16.3553,most_fwver_04.16.3571,most_fwver_04.22.1170,...,most_fwver_05.15.2114,most_fwver_05.15.2120,most_fwver_05.15.2122,most_fwver_05.15.2138,most_fwver_05.15.3104,most_fwver_05.66.3237,most_fwver_05.66.3571,most_fwver_10,most_fwver_10.22.1780,most_fwver_8.5.3
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
29994,44995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29995,44996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29996,44997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[펌웨어 버전별 에러발생 횟수]**

In [12]:
s = pd.pivot_table(error, index = 'user_id', columns = 'fwver', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

fwver,user_id,03.11.1141,03.11.1149,03.11.1167,04.16.2641,04.16.3345,04.16.3439,04.16.3553,04.16.3569,04.16.3571,...,05.15.2120,05.15.2122,05.15.2138,05.15.3104,05.66.3237,05.66.3571,10,10.22.1770,10.22.1780,8.5.3
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,317,0,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,0,0,306,0,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,0,0,0,0,0,1115,0,0,...,0,0,0,0,0,0,0,0,0,0
29994,44995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29995,44996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29996,44997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[펌웨어 버전별 에러발생 비율]**

In [13]:
s = pd.pivot_table(error, index = 'user_id', columns = 'fwver', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
s1 = s.iloc[:,1:].apply(lambda x : x/sum(x), axis = 1)
s2 = pd.concat([s.iloc[:,0], s1], axis = 1)
s_list.append(s2); display(s2)

,user_id,03.11.1141,03.11.1149,03.11.1167,04.16.2641,04.16.3345,04.16.3439,04.16.3553,04.16.3569,04.16.3571,...,05.15.2120,05.15.2122,05.15.2138,05.15.3104,05.66.3237,05.66.3571,10,10.22.1770,10.22.1780,8.5.3
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29994,44995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29995,44996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,44997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### from errtype

**[발생한 에러타입의 수]**

In [14]:
f = error.groupby('user_id')['errtype'].agg([('errtype_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,user_id,errtype_cnt
0,10000,14
1,10001,26
2,10002,16
3,10003,18
4,10004,18
...,...,...
29993,44994,19
29994,44995,16
29995,44996,21
29996,44997,22


**[가장 많이 발생한 에러타입]**

In [15]:
s = error.groupby('user_id')['errtype'].agg([('most_errtype', lambda x : str(x.value_counts().idxmax()))]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

,user_id,most_errtype_1,most_errtype_10,most_errtype_11,most_errtype_12,most_errtype_13,most_errtype_14,most_errtype_15,most_errtype_16,most_errtype_17,...,most_errtype_35,most_errtype_38,most_errtype_39,most_errtype_4,most_errtype_40,most_errtype_41,most_errtype_5,most_errtype_6,most_errtype_7,most_errtype_8
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29994,44995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29995,44996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29996,44997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[에러타입별 에러발생 횟수]**

In [16]:
s = pd.pivot_table(error, index = 'user_id', columns = 'errtype', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

errtype,user_id,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,10000,0,0,8,104,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,0,0,53,1,1,0,0,...,10,18,0,1,1,0,0,113,56,1
2,10002,0,0,2,132,1,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,0,0,2,1,1,0,0,...,8,0,0,1,1,2,0,17,1,0
4,10004,0,0,0,1,0,3,4,0,0,...,16,0,0,1,1,0,0,4,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,0,0,2,12,5,5,0,0,...,16,0,0,0,0,0,0,100,87,0
29994,44995,0,0,0,0,0,2,2,0,0,...,14,1,0,1,1,0,0,12,17,0
29995,44996,0,0,0,0,0,1,1,0,0,...,12,1118,0,1,1,1,0,329,2,3
29996,44997,0,0,0,1,4,2,2,0,0,...,14,0,0,1,1,3,0,30,49,0


**[에러타입별 에러발생 비율]**

In [17]:
s = pd.pivot_table(error, index = 'user_id', columns = 'errtype', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
s1 = s.iloc[:,1:].apply(lambda x : x/sum(x), axis = 1)
s2 = pd.concat([s.iloc[:,0], s1], axis = 1)
s_list.append(s2); display(s2)

,user_id,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,10000,0.0,0.0,0.025237,0.328076,0.000000,0.003155,0.003155,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,10001,0.0,0.0,0.000000,0.000000,0.022410,0.000423,0.000423,0.0,0.0,...,0.004228,0.007611,0.0,0.000423,0.000423,0.000000,0.0,0.047780,0.023679,0.000423
2,10002,0.0,0.0,0.006536,0.431373,0.003268,0.006536,0.003268,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,10003,0.0,0.0,0.000000,0.000000,0.006536,0.003268,0.003268,0.0,0.0,...,0.026144,0.000000,0.0,0.003268,0.003268,0.006536,0.0,0.055556,0.003268,0.000000
4,10004,0.0,0.0,0.000000,0.001287,0.000000,0.003861,0.005148,0.0,0.0,...,0.020592,0.000000,0.0,0.001287,0.001287,0.000000,0.0,0.005148,0.000000,0.002574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0.0,0.0,0.000000,0.001794,0.010762,0.004484,0.004484,0.0,0.0,...,0.014350,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.089686,0.078027,0.000000
29994,44995,0.0,0.0,0.000000,0.000000,0.000000,0.003883,0.003883,0.0,0.0,...,0.027184,0.001942,0.0,0.001942,0.001942,0.000000,0.0,0.023301,0.033010,0.000000
29995,44996,0.0,0.0,0.000000,0.000000,0.000000,0.000448,0.000448,0.0,0.0,...,0.005374,0.500672,0.0,0.000448,0.000448,0.000448,0.0,0.147335,0.000896,0.001343
29996,44997,0.0,0.0,0.000000,0.000041,0.000162,0.000081,0.000081,0.0,0.0,...,0.000567,0.000000,0.0,0.000041,0.000041,0.000122,0.0,0.001216,0.001986,0.000000


### from errcode

**[발생한 에러코드의 수]**

In [18]:
f = error.groupby('user_id')['errcode'].agg([('errcode_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,user_id,errcode_cnt
0,10000,3
1,10001,12
2,10002,4
3,10003,13
4,10004,12
...,...,...
29993,44994,12
29994,44995,8
29995,44996,15
29996,44997,21


**[가장 많이 발생한 에러코드]**

In [19]:
s = error.groupby('user_id')['errcode'].agg([('most_errcode', lambda x : x.value_counts().idxmax())]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

,user_id,most_errcode_0,most_errcode_1,most_errcode_14,most_errcode_2,most_errcode_3,most_errcode_4,most_errcode_5,most_errcode_6,most_errcode_79,...,most_errcode_NFANDROID2,most_errcode_P-44010,most_errcode_S-61001,most_errcode_U-81000,most_errcode_V-21003,most_errcode_V-21008,most_errcode_active,most_errcode_connection timeout,most_errcode_connectionterminated by local host,most_errcode_standby
0,10000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29994,44995,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29995,44996,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29996,44997,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[에러코드별 에러발생 횟수]**

In [20]:
s = pd.pivot_table(error, index = 'user_id', columns = 'errcode', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

errcode,user_id,-1010,-269,-270,0,0001,1,100,10005,10018,...,connection fail to establish,connection timeout,connectionterminated by local host,eDes,http,"me=""",scanning timeout,standby,tVer,terminate by peer user
0,10000,0,0,0,104,0,212,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,0,183,0,1274,0,0,0,...,0,0,0,0,0,0,0,625,0,0
2,10002,0,0,0,132,0,172,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,0,42,0,229,0,0,0,...,0,7,0,0,0,0,0,0,0,0
4,10004,0,0,0,98,0,529,0,0,0,...,1,104,0,0,0,0,0,5,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,0,0,139,0,624,0,0,0,...,7,197,0,0,0,0,0,0,0,0
29994,44995,0,0,0,105,0,370,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29995,44996,0,0,0,251,0,698,0,0,0,...,4,44,0,0,0,0,0,11,0,2
29996,44997,0,0,0,61,0,11373,0,0,0,...,1010,10449,1,0,0,0,3,20,0,1


**[에러코드별 에러발생 비율]**

In [21]:
s = pd.pivot_table(error, index = 'user_id', columns = 'errcode', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
s1 = s.iloc[:,1:].apply(lambda x : x/sum(x), axis = 1)
s2 = pd.concat([s.iloc[:,0], s1], axis = 1)
s_list.append(s2); display(s2)

,user_id,-1010,-269,-270,0,0001,1,100,10005,10018,...,connection fail to establish,connection timeout,connectionterminated by local host,eDes,http,"me=""",scanning timeout,standby,tVer,terminate by peer user
0,10000,0.0,0.0,0.0,0.328076,0.0,0.668770,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
1,10001,0.0,0.0,0.0,0.077378,0.0,0.538689,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.264271,0.0,0.000000
2,10002,0.0,0.0,0.0,0.431373,0.0,0.562092,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
3,10003,0.0,0.0,0.0,0.137255,0.0,0.748366,0.0,0.0,0.0,...,0.000000,0.022876,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
4,10004,0.0,0.0,0.0,0.126126,0.0,0.680824,0.0,0.0,0.0,...,0.001287,0.133848,0.000000,0.0,0.0,0.0,0.000000,0.006435,0.0,0.002574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0.0,0.0,0.0,0.124664,0.0,0.559641,0.0,0.0,0.0,...,0.006278,0.176682,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
29994,44995,0.0,0.0,0.0,0.203883,0.0,0.718447,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
29995,44996,0.0,0.0,0.0,0.112405,0.0,0.312584,0.0,0.0,0.0,...,0.001791,0.019704,0.000000,0.0,0.0,0.0,0.000000,0.004926,0.0,0.000896
29996,44997,0.0,0.0,0.0,0.002473,0.0,0.460987,0.0,0.0,0.0,...,0.040939,0.423534,0.000041,0.0,0.0,0.0,0.000122,0.000811,0.0,0.000041


### from datetime

In [22]:
error['datetime'] = pd.to_datetime(error.time, format = '%Y%m%d%H%M%S')
error['Ymd'] = pd.to_datetime(error.time.apply(lambda x : str(x)[:8]), format = '%Y%m%d')
error['month'] = error.datetime.dt.month
error['day'] = error.datetime.dt.day
error['hour'] = error.datetime.dt.hour
error['dayofweek'] = error.datetime.dt.day_name()

**[에러 발생일 수]**

In [23]:
f = error.groupby('user_id')['datetime'].agg([('error_datetime_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,user_id,error_datetime_cnt
0,10000,285
1,10001,2201
2,10002,284
3,10003,244
4,10004,693
...,...,...
29993,44994,1021
29994,44995,449
29995,44996,1855
29996,44997,24625


**[일별 에러 발생 수]**

In [24]:
f = pd.pivot_table(error, index = 'user_id', columns = 'Ymd', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
f_list.append(f); display(f)

Ymd,user_id,2020-10-31 00:00:00,2020-11-01 00:00:00,2020-11-02 00:00:00,2020-11-03 00:00:00,2020-11-04 00:00:00,2020-11-05 00:00:00,2020-11-06 00:00:00,2020-11-07 00:00:00,2020-11-08 00:00:00,...,2020-11-29 00:00:00,2020-11-30 00:00:00,2020-12-01 00:00:00,2020-12-02 00:00:00,2020-12-03 00:00:00,2020-12-04 00:00:00,2020-12-05 00:00:00,2020-12-09 00:00:00,2020-12-11 00:00:00,2020-12-14 00:00:00
0,10000,0,11,9,18,5,10,9,20,7,...,13,9,0,0,0,0,0,0,0,0
1,10001,0,11,50,29,48,42,29,13,24,...,62,28,0,0,0,0,0,0,0,0
2,10002,0,10,13,13,15,9,8,17,11,...,8,9,0,0,0,0,0,0,0,0
3,10003,0,9,14,10,5,16,5,2,11,...,14,6,0,0,0,0,0,0,0,0
4,10004,0,25,21,49,28,11,28,18,29,...,23,22,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,48,44,28,13,37,40,47,44,...,45,25,0,0,0,0,0,0,0,0
29994,44995,0,16,23,34,10,22,35,24,26,...,11,9,0,0,0,0,0,0,0,0
29995,44996,0,23,16,42,11,22,12,36,26,...,29,29,0,0,0,0,0,0,0,0
29996,44997,0,1214,994,1102,0,0,221,409,947,...,644,499,0,0,0,0,0,0,0,0


**[일별 에러 발생비율]**

In [25]:
f = pd.pivot_table(error, index = 'user_id', columns = 'Ymd', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
f1 = f.iloc[:,1:].apply(lambda x : x/sum(x), axis = 1)
f2 = pd.concat([f.iloc[:,0], f1], axis = 1)
f_list.append(f2); display(f2)

,user_id,2020-10-31 00:00:00,2020-11-01 00:00:00,2020-11-02 00:00:00,2020-11-03 00:00:00,2020-11-04 00:00:00,2020-11-05 00:00:00,2020-11-06 00:00:00,2020-11-07 00:00:00,2020-11-08 00:00:00,...,2020-11-29 00:00:00,2020-11-30 00:00:00,2020-12-01 00:00:00,2020-12-02 00:00:00,2020-12-03 00:00:00,2020-12-04 00:00:00,2020-12-05 00:00:00,2020-12-09 00:00:00,2020-12-11 00:00:00,2020-12-14 00:00:00
0,10000,0.0,0.034700,0.028391,0.056782,0.015773,0.031546,0.028391,0.063091,0.022082,...,0.041009,0.028391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,0.004651,0.021142,0.012262,0.020296,0.017759,0.012262,0.005497,0.010148,...,0.026216,0.011839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,0.0,0.032680,0.042484,0.042484,0.049020,0.029412,0.026144,0.055556,0.035948,...,0.026144,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.029412,0.045752,0.032680,0.016340,0.052288,0.016340,0.006536,0.035948,...,0.045752,0.019608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,0.0,0.032175,0.027027,0.063063,0.036036,0.014157,0.036036,0.023166,0.037323,...,0.029601,0.028314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0.0,0.043049,0.039462,0.025112,0.011659,0.033184,0.035874,0.042152,0.039462,...,0.040359,0.022422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29994,44995,0.0,0.031068,0.044660,0.066019,0.019417,0.042718,0.067961,0.046602,0.050485,...,0.021359,0.017476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29995,44996,0.0,0.010300,0.007165,0.018809,0.004926,0.009852,0.005374,0.016122,0.011644,...,0.012987,0.012987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,44997,0.0,0.049208,0.040290,0.044668,0.000000,0.000000,0.008958,0.016578,0.038385,...,0.026104,0.020226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**[일별 에러발생 누적 수]**

In [26]:
f = pd.pivot_table(error, index = 'user_id', columns = 'Ymd', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
f1 = f.iloc[:,1:].cumsum(axis = 1)
f2 = pd.concat([f.iloc[:,0], f1], axis = 1)
f_list.append(f2); display(f2)

,user_id,2020-10-31 00:00:00,2020-11-01 00:00:00,2020-11-02 00:00:00,2020-11-03 00:00:00,2020-11-04 00:00:00,2020-11-05 00:00:00,2020-11-06 00:00:00,2020-11-07 00:00:00,2020-11-08 00:00:00,...,2020-11-29 00:00:00,2020-11-30 00:00:00,2020-12-01 00:00:00,2020-12-02 00:00:00,2020-12-03 00:00:00,2020-12-04 00:00:00,2020-12-05 00:00:00,2020-12-09 00:00:00,2020-12-11 00:00:00,2020-12-14 00:00:00
0,10000,0,11,20,38,43,53,62,82,89,...,308,317,317,317,317,317,317,317,317,317
1,10001,0,11,61,90,138,180,209,222,246,...,2337,2365,2365,2365,2365,2365,2365,2365,2365,2365
2,10002,0,10,23,36,51,60,68,85,96,...,297,306,306,306,306,306,306,306,306,306
3,10003,0,9,23,33,38,54,59,61,72,...,300,306,306,306,306,306,306,306,306,306
4,10004,0,25,46,95,123,134,162,180,209,...,755,777,777,777,777,777,777,777,777,777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,48,92,120,133,170,210,257,301,...,1090,1115,1115,1115,1115,1115,1115,1115,1115,1115
29994,44995,0,16,39,73,83,105,140,164,190,...,506,515,515,515,515,515,515,515,515,515
29995,44996,0,23,39,81,92,114,126,162,188,...,2204,2233,2233,2233,2233,2233,2233,2233,2233,2233
29996,44997,0,1214,2208,3310,3310,3310,3531,3940,4887,...,24172,24671,24671,24671,24671,24671,24671,24671,24671,24671


**[일별 에러발생 누적비율]**

In [27]:
f = pd.pivot_table(error, index = 'user_id', columns = 'Ymd', values = 'time', aggfunc = 'count', fill_value = 0).reset_index()
f1 = f.iloc[:,1:].cumsum(axis = 1).apply(lambda x : x/max(x), axis = 1)
f2 = pd.concat([f.iloc[:,0], f1], axis = 1)
f_list.append(f2); display(f2)

,user_id,2020-10-31 00:00:00,2020-11-01 00:00:00,2020-11-02 00:00:00,2020-11-03 00:00:00,2020-11-04 00:00:00,2020-11-05 00:00:00,2020-11-06 00:00:00,2020-11-07 00:00:00,2020-11-08 00:00:00,...,2020-11-29 00:00:00,2020-11-30 00:00:00,2020-12-01 00:00:00,2020-12-02 00:00:00,2020-12-03 00:00:00,2020-12-04 00:00:00,2020-12-05 00:00:00,2020-12-09 00:00:00,2020-12-11 00:00:00,2020-12-14 00:00:00
0,10000,0.0,0.034700,0.063091,0.119874,0.135647,0.167192,0.195584,0.258675,0.280757,...,0.971609,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,10001,0.0,0.004651,0.025793,0.038055,0.058351,0.076110,0.088372,0.093869,0.104017,...,0.988161,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,10002,0.0,0.032680,0.075163,0.117647,0.166667,0.196078,0.222222,0.277778,0.313725,...,0.970588,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,10003,0.0,0.029412,0.075163,0.107843,0.124183,0.176471,0.192810,0.199346,0.235294,...,0.980392,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,10004,0.0,0.032175,0.059202,0.122265,0.158301,0.172458,0.208494,0.231660,0.268983,...,0.971686,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0.0,0.043049,0.082511,0.107623,0.119283,0.152466,0.188341,0.230493,0.269955,...,0.977578,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
29994,44995,0.0,0.031068,0.075728,0.141748,0.161165,0.203883,0.271845,0.318447,0.368932,...,0.982524,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
29995,44996,0.0,0.010300,0.017465,0.036274,0.041200,0.051052,0.056426,0.072548,0.084192,...,0.987013,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
29996,44997,0.0,0.049208,0.089498,0.134166,0.134166,0.134166,0.143124,0.159702,0.198087,...,0.979774,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


**[요일별 에러 발생비율]**

In [28]:
f = error.groupby('user_id')['datetime'].agg([('mon_error_prob', lambda x : np.mean(x.dt.dayofweek.isin([0]))),
                                              ('tue_error_prob', lambda x : np.mean(x.dt.dayofweek.isin([1]))),
                                              ('wen_error_prob', lambda x : np.mean(x.dt.dayofweek.isin([2]))),
                                              ('thu_error_prob', lambda x : np.mean(x.dt.dayofweek.isin([3]))),
                                              ('fri_error_prob', lambda x : np.mean(x.dt.dayofweek.isin([4]))),
                                              ('sat_error_prob', lambda x : np.mean(x.dt.dayofweek.isin([5]))),
                                              ('sun_error_prob', lambda x : np.mean(x.dt.dayofweek.isin([6])))]).reset_index()
f_list.append(f); display(f)

,user_id,mon_error_prob,tue_error_prob,wen_error_prob,thu_error_prob,fri_error_prob,sat_error_prob,sun_error_prob
0,10000,0.116719,0.129338,0.141956,0.141956,0.135647,0.170347,0.164038
1,10001,0.069345,0.058351,0.045243,0.070190,0.648626,0.049471,0.058774
2,10002,0.169935,0.111111,0.150327,0.130719,0.111111,0.137255,0.189542
3,10003,0.166667,0.169935,0.137255,0.147059,0.120915,0.071895,0.186275
4,10004,0.123552,0.185328,0.148005,0.075933,0.220077,0.108108,0.138996
...,...,...,...,...,...,...,...,...
29993,44994,0.158744,0.119283,0.135426,0.124664,0.138117,0.104036,0.219731
29994,44995,0.161165,0.166990,0.106796,0.132039,0.143689,0.137864,0.151456
29995,44996,0.099866,0.091357,0.075683,0.121809,0.184953,0.187640,0.238692
29996,44997,0.135422,0.136760,0.116453,0.174253,0.084674,0.176969,0.175469


**[월별 에러 발생비율]**

In [29]:
s = error.groupby('user_id')['datetime'].agg([('oct_error_prob', lambda x: np.mean(x.dt.month.isin([10]))),
                                              ('nov_error_prob', lambda x: np.mean(x.dt.month.isin([11]))),
                                              ('dec_error_prob', lambda x: np.mean(x.dt.month.isin([12])))]).reset_index()
s_list.append(s); display(s)

,user_id,oct_error_prob,nov_error_prob,dec_error_prob
0,10000,0.0,1.0,0.0
1,10001,0.0,1.0,0.0
2,10002,0.0,1.0,0.0
3,10003,0.0,1.0,0.0
4,10004,0.0,1.0,0.0
...,...,...,...,...
29993,44994,0.0,1.0,0.0
29994,44995,0.0,1.0,0.0
29995,44996,0.0,1.0,0.0
29996,44997,0.0,1.0,0.0


**[시간대별 에러 발생비율]**

In [30]:
f = error.groupby('user_id')['datetime'].agg([('0_error_prob', lambda x : np.mean(x.dt.hour.isin([0]))),
                                              ('1_error_prob', lambda x : np.mean(x.dt.hour.isin([1]))),
                                              ('2_error_prob', lambda x : np.mean(x.dt.hour.isin([2]))),
                                              ('3_error_prob', lambda x : np.mean(x.dt.hour.isin([3]))),
                                              ('4_error_prob', lambda x : np.mean(x.dt.hour.isin([4]))),
                                              ('5_error_prob', lambda x : np.mean(x.dt.hour.isin([5]))),
                                              ('6_error_prob', lambda x : np.mean(x.dt.hour.isin([6]))),
                                              ('7_error_prob', lambda x : np.mean(x.dt.hour.isin([7]))),
                                              ('8_error_prob', lambda x : np.mean(x.dt.hour.isin([8]))),
                                              ('9_error_prob', lambda x : np.mean(x.dt.hour.isin([9]))),
                                              ('10_error_prob', lambda x : np.mean(x.dt.hour.isin([10]))),
                                              ('11_error_prob', lambda x : np.mean(x.dt.hour.isin([11]))),
                                              ('12_error_prob', lambda x : np.mean(x.dt.hour.isin([12]))),
                                              ('13_error_prob', lambda x : np.mean(x.dt.hour.isin([13]))),
                                              ('14_error_prob', lambda x : np.mean(x.dt.hour.isin([14]))),
                                              ('15_error_prob', lambda x : np.mean(x.dt.hour.isin([15]))),
                                              ('16_error_prob', lambda x : np.mean(x.dt.hour.isin([16]))),
                                              ('17_error_prob', lambda x : np.mean(x.dt.hour.isin([17]))),
                                              ('18_error_prob', lambda x : np.mean(x.dt.hour.isin([18]))),
                                              ('19_error_prob', lambda x : np.mean(x.dt.hour.isin([19]))),
                                              ('20_error_prob', lambda x : np.mean(x.dt.hour.isin([20]))),
                                              ('21_error_prob', lambda x : np.mean(x.dt.hour.isin([21]))),
                                              ('22_error_prob', lambda x : np.mean(x.dt.hour.isin([22]))),
                                              ('23_error_prob', lambda x : np.mean(x.dt.hour.isin([23])))]).reset_index()
f_list.append(f); display(f)

,user_id,0_error_prob,1_error_prob,2_error_prob,3_error_prob,4_error_prob,5_error_prob,6_error_prob,7_error_prob,8_error_prob,...,14_error_prob,15_error_prob,16_error_prob,17_error_prob,18_error_prob,19_error_prob,20_error_prob,21_error_prob,22_error_prob,23_error_prob
0,10000,0.104101,0.041009,0.037855,0.088328,0.053628,0.141956,0.088328,0.050473,0.034700,...,0.031546,0.028391,0.018927,0.047319,0.059937,0.037855,0.000000,0.031546,0.015773,0.006309
1,10001,0.026216,0.014376,0.028753,0.011416,0.008879,0.000000,0.000846,0.009302,0.012262,...,0.077378,0.081607,0.088795,0.064693,0.035941,0.097252,0.074419,0.109091,0.029598,0.019027
2,10002,0.013072,0.009804,0.039216,0.081699,0.019608,0.068627,0.055556,0.022876,0.032680,...,0.013072,0.088235,0.016340,0.019608,0.016340,0.091503,0.078431,0.098039,0.042484,0.032680
3,10003,0.104575,0.049020,0.026144,0.058824,0.120915,0.026144,0.029412,0.147059,0.000000,...,0.000000,0.026144,0.009804,0.016340,0.016340,0.062092,0.127451,0.098039,0.006536,0.009804
4,10004,0.009009,0.000000,0.006435,0.171171,0.000000,0.015444,0.011583,0.099099,0.038610,...,0.010296,0.009009,0.023166,0.028314,0.088803,0.142857,0.055341,0.039897,0.090090,0.038610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0.001794,0.000000,0.001794,0.018834,0.073543,0.004484,0.009865,0.008969,0.012556,...,0.090583,0.033184,0.079821,0.064574,0.086996,0.041256,0.055605,0.114798,0.078924,0.017040
29994,44995,0.015534,0.013592,0.159223,0.017476,0.013592,0.015534,0.001942,0.000000,0.005825,...,0.038835,0.023301,0.027184,0.027184,0.056311,0.052427,0.114563,0.034951,0.062136,0.040777
29995,44996,0.043439,0.008509,0.009404,0.008957,0.008061,0.043439,0.002687,0.090909,0.009404,...,0.012987,0.050605,0.081953,0.046126,0.046126,0.122257,0.057770,0.073444,0.084639,0.022391
29996,44997,0.041830,0.037331,0.032954,0.031697,0.028049,0.022374,0.024361,0.026874,0.030076,...,0.045560,0.044465,0.041101,0.056504,0.061246,0.065543,0.055004,0.062584,0.049167,0.061287


**[가장 에러가 많이 발생한 요일]**

In [31]:
s = error.groupby('user_id')['dayofweek'].agg([('most_dayofweek', lambda x : x.value_counts().idxmax())]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

,user_id,most_dayofweek_Friday,most_dayofweek_Monday,most_dayofweek_Saturday,most_dayofweek_Sunday,most_dayofweek_Thursday,most_dayofweek_Tuesday,most_dayofweek_Wednesday
0,10000,0,0,1,0,0,0,0
1,10001,1,0,0,0,0,0,0
2,10002,0,0,0,1,0,0,0
3,10003,0,0,0,1,0,0,0
4,10004,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
29993,44994,0,0,0,1,0,0,0
29994,44995,0,0,0,0,0,1,0
29995,44996,0,0,0,1,0,0,0
29996,44997,0,0,1,0,0,0,0


**[가장 에러가 많이 발생한 시간대]**

In [32]:
s = error.groupby('user_id')['hour'].agg([('most_hour', lambda x : x.value_counts().idxmax())]).reset_index()
s['most_hour'] = s['most_hour'].astype(str)
s = pd.get_dummies(s)
s_list.append(s); display(s)

,user_id,most_hour_0,most_hour_1,most_hour_10,most_hour_11,most_hour_12,most_hour_13,most_hour_14,most_hour_15,most_hour_16,...,most_hour_21,most_hour_22,most_hour_23,most_hour_3,most_hour_4,most_hour_5,most_hour_6,most_hour_7,most_hour_8,most_hour_9
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,10001,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,44994,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
29994,44995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29995,44996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29996,44997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 추가 feature

**[총 에러 발생 수]**

In [33]:
f = error.groupby('user_id')['datetime'].agg([('error_cnt_sum','count')]).reset_index()
f_list.append(f); display(f)

,user_id,error_cnt_sum
0,10000,317
1,10001,2365
2,10002,306
3,10003,306
4,10004,777
...,...,...
29993,44994,1115
29994,44995,515
29995,44996,2233
29996,44997,24671


# Quality

In [34]:
quality = pd.concat([train_quality, test_quality])
q2 = quality.iloc[:,3:].applymap(lambda x : float(str(x).replace(',','')))
quality = pd.concat([quality.iloc[:,:3], q2], axis = 1)
quality['datetime'] = pd.to_datetime(quality['time'], format = '%Y%m%d%H%M%S')
quality['Ymd'] = pd.to_datetime(quality.time.apply(lambda x : str(x)[:8]), format = '%Y%m%d')
quality['month'] = quality.datetime.dt.month
quality['day'] = quality.datetime.dt.day
quality['hour'] = quality.datetime.dt.hour
quality['dayofweek'] = quality.datetime.dt.day_name()
quality

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,...,quality_9,quality_10,quality_11,quality_12,datetime,Ymd,month,day,hour,dayofweek
0,20201129090000,10000,05.15.2138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,2020-11-29,11,29,9,Sunday
1,20201129090000,10000,05.15.2138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,2020-11-29,11,29,9,Sunday
2,20201129090000,10000,05.15.2138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,2020-11-29,11,29,9,Sunday
3,20201129090000,10000,05.15.2138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,2020-11-29,11,29,9,Sunday
4,20201129090000,10000,05.15.2138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,2020-11-29,11,29,9,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,20201125180000,44997,04.22.1750,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,3.0,0.0,0.0,2020-11-25 18:00:00,2020-11-25,11,25,18,Wednesday
747968,20201125180000,44997,04.22.1750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,2020-11-25 18:00:00,2020-11-25,11,25,18,Wednesday
747969,20201125180000,44997,04.22.1750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,2020-11-25 18:00:00,2020-11-25,11,25,18,Wednesday
747970,20201125180000,44997,04.22.1750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,2020-11-25 18:00:00,2020-11-25,11,25,18,Wednesday


**[퀄리티별 에러발생 횟수]**

In [35]:
q1 = quality.iloc[:,3:16].applymap(lambda x : 0 if x == 0.0 else 1)
q2 = pd.concat([quality.iloc[:,1], q1], axis = 1)
s = q2.groupby('user_id')[['quality_0','quality_1','quality_2','quality_3','quality_4','quality_5','quality_6',
                           'quality_7','quality_8','quality_9','quality_10','quality_11','quality_12']].sum().reset_index()
s.columns = ['user_id','quality_0_cnt','quality_1_cnt','quality_2_cnt','quality_3_cnt',
             'quality_4_cnt','quality_5_cnt','quality_6_cnt','quality_7_cnt','quality_8_cnt',
             'quality_9_cnt','quality_10_cnt','quality_11_cnt','quality_12_cnt']
s_list.append(s); display(s)

,user_id,quality_0_cnt,quality_1_cnt,quality_2_cnt,quality_3_cnt,quality_4_cnt,quality_5_cnt,quality_6_cnt,quality_7_cnt,quality_8_cnt,quality_9_cnt,quality_10_cnt,quality_11_cnt,quality_12_cnt
0,10000,0,0,0,0,0,2,0,0,0,0,24,0,0
1,10002,3,2,3,0,0,14,5,24,0,12,84,2,0
2,10004,2,2,2,0,0,5,3,12,0,0,24,2,0
3,10005,10,10,10,0,0,16,12,12,0,0,24,10,0
4,10006,0,0,0,0,0,3,1,12,0,0,36,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16544,44990,4,4,4,0,0,13,4,0,0,0,36,4,0
16545,44993,5,2,5,0,0,7,6,24,0,36,36,2,0
16546,44994,11,11,11,0,0,12,11,0,0,0,12,11,0
16547,44996,5,5,5,0,0,20,5,0,0,0,108,5,0


**[datetime별 에러발생 일 수]**

In [36]:
f = quality.groupby('user_id')['datetime'].agg([('quality_datetime_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,user_id,quality_datetime_cnt
0,10000,2
1,10002,8
2,10004,2
3,10005,2
4,10006,3
...,...,...
16544,44990,3
16545,44993,4
16546,44994,1
16547,44996,9


**[퀄리티별 sum]**

In [37]:
s = quality.groupby('user_id')[['quality_0','quality_1','quality_2','quality_3','quality_4','quality_5','quality_6',
                                'quality_7','quality_8','quality_9','quality_10','quality_11','quality_12']].sum().reset_index()
s.columns = ['user_id','quality_0_sum','quality_1_sum','quality_2_sum','quality_3_sum','quality_4_sum','quality_5_sum','quality_6_sum',
             'quality_7_sum','quality_8_sum','quality_9_sum','quality_10_sum','quality_11_sum','quality_12_sum']
s_list.append(s); display(s)

,user_id,quality_0_sum,quality_1_sum,quality_2_sum,quality_3_sum,quality_4_sum,quality_5_sum,quality_6_sum,quality_7_sum,quality_8_sum,quality_9_sum,quality_10_sum,quality_11_sum,quality_12_sum
0,10000,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0
1,10002,0.0,-2.0,-1.0,0.0,0.0,29.0,44.0,552.0,0.0,12.0,372.0,-2.0,0.0
2,10004,-2.0,-2.0,-2.0,0.0,0.0,2.0,85.0,1044.0,0.0,0.0,48.0,-2.0,0.0
3,10005,-10.0,-10.0,-10.0,0.0,0.0,0.0,26.0,432.0,0.0,0.0,120.0,-10.0,0.0
4,10006,0.0,0.0,0.0,0.0,0.0,12.0,4.0,48.0,0.0,0.0,144.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16544,44990,-4.0,-4.0,-4.0,0.0,0.0,46.0,-4.0,0.0,0.0,0.0,600.0,-4.0,0.0
16545,44993,7.0,-2.0,7.0,0.0,0.0,12.0,107.0,1308.0,0.0,108.0,168.0,-2.0,0.0
16546,44994,-11.0,-11.0,-11.0,0.0,0.0,-9.0,-11.0,0.0,0.0,0.0,24.0,-11.0,0.0
16547,44996,-5.0,-5.0,-5.0,0.0,0.0,32.0,-5.0,0.0,0.0,0.0,444.0,-5.0,0.0


**[퀄리티 발생 횟수]**

In [38]:
f = quality.groupby('user_id')['datetime'].agg([('quality_cnt','count')]).reset_index()
f_list.append(f); display(f)

,user_id,quality_cnt
0,10000,24
1,10002,96
2,10004,24
3,10005,24
4,10006,36
...,...,...
16544,44990,36
16545,44993,48
16546,44994,12
16547,44996,108


# Feature Merge

### numeric feature merge

In [39]:
feature = pd.DataFrame({'user_id' : list(range(10000,25000)) + list(range(30000,44999))})
for f in f_list:
    feature = pd.merge(feature, f, on='user_id', how='left')    
feature = feature.fillna(0)

### sparse feature merge

In [40]:
sparse = pd.DataFrame({'user_id' : list(range(10000,25000)) + list(range(30000,44999))})
for s in s_list:
    sparse = pd.merge(sparse, s, on='user_id', how='left')    
sparse = sparse.fillna(0)

In [41]:
features = pd.merge(feature, sparse, on = 'user_id', how = 'left')

### Feature Scaling

In [42]:
feature = features.apply(lambda x : x.clip(x.quantile(0.05), x.quantile(0.95)), axis = 0)
col_name = list(feature.columns)

In [43]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

ss = StandardScaler()
mms = MinMaxScaler()

feature = ss.fit_transform(feature)
feature = mms.fit_transform(feature)

feature = pd.DataFrame(feature, columns = [str(i) for i in col_name])

# train_feature = feature.iloc[:15000, :]
# test_feature = feature.iloc[15000:, :]

In [44]:
def getPCA(df):
    max_col = df.shape[1]
    pca = PCA(n_components=max_col, random_state=0).fit(df)

    cumsum = np.cumsum(pca.explained_variance_ratio_) 
    num_col = np.argmax(cumsum >= 0.99) + 1 

    pca = PCA(n_components = num_col, random_state=0).fit_transform(df)
    return pd.DataFrame(pca)

In [47]:
from sklearn.decomposition import PCA
features = getPCA(feature) ;features

MemoryError: Unable to allocate 2.07 GiB for an array with shape (29999, 9263) and data type float64

# Feature to CSV

In [ ]:
train_x = features.iloc[:15000,:]
test_x = features.iloc[15000:,:]

In [ ]:
train_x.to_csv('train_x_9263.csv',index = False)
test_x.to_csv('test_x_9263.csv',index = False)